## ResNet을 활용하여 모델 구성하기

In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model

# ImageNet으로 학습된 모델을 불러옵니다.
# 각 인자에 대한 설명은 5.4절을 참고하세요.
# inception_v3 = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (32, 32, 3))
base_model = ResNet50(weights = 'imagenet', include_top = False)

# 임의로 전처리 layer를 함수 파라미터로 전달하겠습니다.
def get_model(num_classes, norm_layer):
    inputs = Input(shape = (32, 32, 3))
    norm_inputs = norm_layer(inputs)
    
    x = base_model(norm_inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation = 'softmax')(x)
    
    # 모델을 생성합니다.
    model = Model(inputs = inputs, outputs = x)
    
    return model

# 모델 구조를 확인해보세요!
# 또는 plot_model()로 그려보아도 좋습니다.
# model.summary()

## CIFAR-10 학습시켜보기

In [ ]:
from tensorflow.keras.datasets import cifar10
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

norm_layer = tf.keras.layers.Normalization(axis = -1, dtype = 'float32')
norm_layer.adapt(x_train)

model = get_model(num_classes=10, norm_layer = norm_layer)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])
print('model complete...')

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

print(x_train.shape, len(y_train))
print(x_val.shape, len(y_val))

# 학습을 진행합니다.
history = model.fit(x_train, y_train,
                    epochs = 30,
                    batch_size = 32,
                    validation_data = (x_val, y_val))